In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import sqlite3

In [4]:
conn = sqlite3.connect("../database.db")

df = pd.read_sql("SELECT * FROM Ticker_1H", con=conn)
df.head()

,Datetime,Open,High,Low,Close,Volume,Ticker,InsertTime
0,2024-01-16 09:30:00.000000,182.500000,182.850006,180.934006,182.759995,16725135,AAPL,2026-01-13 11:04:55.902870
1,2024-01-16 10:30:00.000000,182.770004,184.259995,182.520004,183.949997,11408997,AAPL,2026-01-13 11:04:55.902870
2,2024-01-16 11:30:00.000000,183.960007,184.160004,182.380005,182.414993,5404103,AAPL,2026-01-13 11:04:55.902870
3,2024-01-16 12:30:00.000000,182.419998,182.929993,182.210007,182.735001,4292807,AAPL,2026-01-13 11:04:55.902870
4,2024-01-16 13:30:00.000000,182.750000,182.949905,182.139999,182.458603,6561011,AAPL,2026-01-13 11:04:55.902870


In [6]:
df.Ticker.drop_duplicates().to_numpy()

array(['AAPL', 'MSFT', 'CSCO', 'TSLA', '^GSPC', '^SP400', '^SP600', 'DUK',
       'KO', 'KMI', 'BP', '600182.SS', 'F', 'TM', 'NMM', 'XPO'],
      dtype=object)